In [18]:
import numpy as np
import json
import os
from benchmark.run import args
from libem.prepare.datasets import abt_buy
from libem.cascade.function import online
from util import sensitivity_analysis, generate_stats, confidence_cost_plot, confidence_f1_plot, plot_result, save_results

In [19]:
cascade_file = "/Users/charlesxu/libem/examples/cascade/output/2024-10-18-12-40-37-beer-cascade.json"
prematch_file = "/Users/charlesxu/libem/examples/cascade/output/2024-10-18-12-40-37-beer-gpt-4o-mini.json"
match_file = "/Users/charlesxu/libem/examples/cascade/output/2024-10-18-12-40-37-beer-gpt-4o.json"

with open(cascade_file, 'r') as file:
    cascade_stats = json.load(file)

with open(prematch_file, 'r') as file:
    prematch_single = json.load(file)
    
with open(match_file, 'r') as file:
    match_single = json.load(file)

In [ ]:
# for data in [cascade_stats, prematch_single, match_single]:
#     cost = data['stats']['tokens']['cost']
#     print(cost)

# for model_name, data in zip([prematch_model, match_model, "Cascade"], [prematch_single, match_single, cascade]):
#     metrics_data['f1'][model_name] = data['stats']['f1']
#     metrics_data['cost'][model_name] = data['stats']['tokens']['cost']
    
# normalized_metrics = {
#     'cost': cost
# }
# metric = 'latency'
# for metric, values in normalized_metrics.items():
#         plt.figure(figsize=(8, 5))
#         plt.bar(np.arange(len(approaches)), values, bar_width, color=metric_colors[metric], label=metric_labels[metric])
#         plt.xticks(np.arange(len(approaches)), approaches)
#         plt.xlabel('Models')
#         plt.ylabel(metric_labels[metric])
#         plt.title(metric_titles[metric])
#         plt.legend()
#         plt.tight_layout()
#         file_name = f"normalized-{metric}-comparison-{timestamp}.png"
#         # plt.savefig(os.path.join(output_path, file_name))
#         plt.show()
output_path = "/Users/charlesxu/libem/examples/cascade/examples/cascade/output/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
plot_result(cascade_stats, prematch_single, match_single)

In [22]:
prematch_results = prematch_single['results']
match_results = match_single['results']

In [23]:
prematch_mismatches = [pair for pair in prematch_results if (pair['label'] == 1 and "no" in pair['pred']) or (pair['label'] == 0 and "yes" in pair['pred'])]

match_dict = {f"{pair['left']['beer_name']}-{pair['right']['beer_name']}": pair for pair in match_results}

combined_mismatches = []

for prematch in prematch_mismatches:
    pair_id = f"{prematch['left']['beer_name']}-{prematch['right']['beer_name']}"
    match = match_dict.get(pair_id)
    if match:
        combined_mismatches.append({
            "pair_id": pair_id,
            "prematch": {
                "pred": prematch['pred'],
                "confidence": prematch['confidence'],
                "label": prematch['label']
            },
            "match": {
                "pred": match['pred'],
                "confidence": match['confidence'],
                "label": match['label']
            }
        })

with open("mismatches.json", "w") as file:
    json.dump(combined_mismatches, file, indent=4)